## LSTM for binary sentiment classification, on MNIST

- Classify MNIST data with LSTM
- Use 28 timesteps


In [2]:
## Single LSTM Layer

### Step 1: Import modules
import os
import logging

import numpy as np
import keras
import keras.backend as K

from keras.datasets import mnist
from keras.models import Model, Input, load_model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM

## Fix random seed for reproducibility
np.random.seed(20170704)


## Check proper working directory
#os.chdir('path/to/day_2/')
#if os.getcwd().split('/')[-1] == 'day_2':
#    pass
#else:
#    raise OSError('Check current working directory.\n'
#                  'If not specified as instructed, '
#                  'more errors will occur throught the code.\n'
#                  '- Current working directory: %s' % os.getcwd())
## Check proper working directory
path = os.getcwd()
os.chdir(path)
if os.getcwd().split('/')[-1] == 'DLdata':
    pass
else:
    path = os.getcwd()+'/DLdata'
    #raise OSError('Check current working directory.\n'
    #              'If not specified as instructed, '
    #              'more errors will occur throught the code.\n'
    #              '- Current working directory: %s' % os.getcwd())
print(path)


## Set logging
def set_logging(testlog=False):
    # 1. Make 'logger' instance
    logger = logging.getLogger()
    # 2. Make 'formatter'
    formatter = logging.Formatter(
            '[%(levelname)s:%(lineno)s] %(asctime)s > %(message)s'
            )
    # 3. Make 'streamHandler'
    streamHandler = logging.StreamHandler()
    # 4. Set 'formatter' to 'streamHandler'
    streamHandler.setFormatter(formatter)
    # 5. Add streamHandler to 'logger' instance
    logger.addHandler(streamHandler)
    # 6. Set level of log; DEBUG -> INFO -> WARNING -> ERROR -> CRITICAL
    logger.setLevel(logging.DEBUG)
    # 7. Print test INFO message
    if testlog: # default is 'False'
        logging.info("Stream logging available!")
    
    return logger

_ = set_logging()


####################################################################################


### Step 2: Load & preprocess data

## 2-1. Load
# Data, shuffled and split between train / test sets
# shape of X_train, X_test; (batch_size, height, width)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
logging.info('MNIST data has been loaded.')

## 2-2. Preprocess
# Change data types to 'float32'
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


# Normalization
X_train /= 255
X_test /= 255

# Check input shape of data; (batch_size, timesteps, input_dim)
timesteps = 28
input_dim = 28
#timesteps = 14
#input_dim = 56

X_train = X_train.reshape(-1, timesteps, input_dim) # -1: reshape 할때, 알아서 채우라는 의미
X_test = X_test.reshape(-1, timesteps, input_dim)

logging.info('final training data shape: {}'.format(X_train.shape))
logging.info('final test data shape: {}'.format(X_test.shape))

# Convert class vectors to binary class matrices (one-hot vectors)
num_classes = 10
Y_train = keras.utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.to_categorical(y_test, num_classes)

logging.info('final train label shape: {}'.format(y_train.shape))
logging.info('final test label shape: {}'.format(y_test.shape))


####################################################################################


### Step 3: Build Model

## 3-1. Define hyperparameters
epochs = 10
batch_size = 128
hidden_size = 100
num_classes = 10

## 3-2. Define RNN model with LSTM cells for MNIST data

# TODO: DEFINE INPUT TENSOR (hint; (timesteps, input_dim))
input_sequences = Input(shape=(timesteps, input_dim), name='input_sequence')

# TODO: DEFINE LSTM LAYER (with hidden_size=100)
x = LSTM(units=hidden_size,
         dropout=0.,
         recurrent_dropout=0.,
         kernel_initializer='glorot_uniform',
         recurrent_initializer='orthogonal',
         return_sequences=False,
         name='lstm')(input_sequences)

# TODO: DEFINE DENSE LAYER (with hidden_size=100)
x = Dense(units=100, activation='relu', name='fc')(x)

# TODO: DEFINE SOFTMAX LAYER (10 classes)
prediction = Dense(units=10, activation='softmax', name='prediction')(x)

# INSTANTIATE MODEL
model = Model(inputs=input_sequences,
              outputs=prediction,
              name='LSTM_mnist')


####################################################################################


### Step 4: Define callbacks

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

# List of callbacks
callbacks = []

# Model checkpoints
ckpt_path = path+'/lstm_mnist_ckpts/lstm_mnist.{epoch:02d}-{val_acc:.2f}.hdf5'
if not os.path.exists(os.path.dirname(ckpt_path)):
    os.makedirs(os.path.dirname(ckpt_path))

checkpoint = ModelCheckpoint(filepath=ckpt_path,
                             monitor='val_acc',
                             save_best_only=True,
                             verbose=1)
callbacks.append(checkpoint)

# Stop training early
earlystopping = EarlyStopping(monitor='val_loss',
                              patience=5,
                              verbose=1)
callbacks.append(earlystopping)

# Reduce learning rate when learning does not improve
reducelr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.1, 
                             patience=10,
                             verbose=1)
callbacks.append(reducelr)

# Tensorboard for visualization; only available with tensorflow backend
# In the terminal; tensorboard --logdir='/full/path/to/lstm_mnist_logs/'
if K.backend() == 'tensorflow':
    logging.info('Using tensorboard callback')
    tb_logdir = path+'/lstm_mnist_logs/'
    if not os.path.exists(tb_logdir):
        os.makedirs(tb_logdir)
    tensorboard = TensorBoard(log_dir=tb_logdir,
                              histogram_freq=1,
                              write_graph=True)
    callbacks.append(tensorboard)


####################################################################################


### Step 5: Compile & train model

# TODO: COMPILE MODEL
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])

print(model.summary())


history = model.fit(X_train, Y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=callbacks,
                    verbose=1)


####################################################################################


### Step 6: Save & load model weights

# Save model weights
model.save_weights(path+'/weights/lstm_mnist_weights.h5')

# Load model weights
model.load_weights(path+'/weights/lstm_mnist_weights.h5')


####################################################################################


### Step 7: Test model performance

test_scores = model.evaluate(X_test, Y_test, verbose=1)
logging.info('Test accuracy: %.2f%%' %(test_scores[1] * 100))
#print("Test accuracy: %.2f%%" % (test_scores[1] * 100))

#train_scores = model.evaluate(X_train, y_train, verbose=1)
#print("Train accuracy: %.2f%%" % (train_scores[1] * 100))


####################################################################################


### Step 8: Using best checkpoint model

best_model_path = path+'/lstm_mnist_ckpts/lstm_mnist.08-1.00.hdf5' # must change filename
best_model = load_model(best_model_path)
best_model.summary()
test_scores = best_model.evaluate(X_test, Y_test, verbose=1)
logging.info('Test accuracy: %.2f%%' %(test_scores[1] * 100))

K.clear_session()

[INFO:80] 2017-07-06 00:59:54,449 > MNIST data has been loaded.
[INFO:80] 2017-07-06 00:59:54,449 > MNIST data has been loaded.


/home/user/DataScience/DataScience/Study Note/Deep Learning/DLdata


[INFO:101] 2017-07-06 00:59:54,534 > final training data shape: (60000, 28, 28)
[INFO:101] 2017-07-06 00:59:54,534 > final training data shape: (60000, 28, 28)
[INFO:102] 2017-07-06 00:59:54,535 > final test data shape: (10000, 28, 28)
[INFO:102] 2017-07-06 00:59:54,535 > final test data shape: (10000, 28, 28)
[INFO:109] 2017-07-06 00:59:54,537 > final train label shape: (60000,)
[INFO:109] 2017-07-06 00:59:54,537 > final train label shape: (60000,)
[INFO:110] 2017-07-06 00:59:54,538 > final test label shape: (10000,)
[INFO:110] 2017-07-06 00:59:54,538 > final test label shape: (10000,)
[INFO:190] 2017-07-06 00:59:54,657 > Using tensorboard callback
[INFO:190] 2017-07-06 00:59:54,657 > Using tensorboard callback


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sequence (InputLayer)  (None, 28, 28)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               51600     
_________________________________________________________________
fc (Dense)                   (None, 100)               10100     
_________________________________________________________________
prediction (Dense)           (None, 10)                1010      
Total params: 62,710
Trainable params: 62,710
Non-trainable params: 0
_________________________________________________________________
None
Train on 54000 samples, validate on 6000 samples
INFO:tensorflow:Summary name lstm_2/kernel:0 is illegal; using lstm_2/kernel_0 instead.


[INFO:82] 2017-07-06 00:59:55,457 > Summary name lstm_2/kernel:0 is illegal; using lstm_2/kernel_0 instead.
[INFO:82] 2017-07-06 00:59:55,457 > Summary name lstm_2/kernel:0 is illegal; using lstm_2/kernel_0 instead.


...

[INFO:82] 2017-07-06 00:59:55,471 > Summary name prediction/bias:0 is illegal; using prediction/bias_0 instead.
[INFO:82] 2017-07-06 00:59:55,471 > Summary name prediction/bias:0 is illegal; using prediction/bias_0 instead.


Epoch 1/10
54000/54000 [==============================] - 10s - loss: 0.1133 - acc: 0.9582 - val_loss: 0.0474 - val_acc: 0.9844
Epoch 2/10
54000/54000 [==============================] - 10s - loss: 0.0376 - acc: 0.9870 - val_loss: 0.0222 - val_acc: 0.9925
Epoch 3/10
54000/54000 [==============================] - 10s - loss: 0.0247 - acc: 0.9918 - val_loss: 0.0164 - val_acc: 0.9946
Epoch 4/10
54000/54000 [==============================] - 10s - loss: 0.0179 - acc: 0.9942 - val_loss: 0.0134 - val_acc: 0.9953
Epoch 5/10
54000/54000 [==============================] - 10s - loss: 0.0144 - acc: 0.9952 - val_loss: 0.0121 - val_acc: 0.9958
Epoch 6/10
54000/54000 [==============================] - 10s - loss: 0.0115 - acc: 0.9962 - val_loss: 0.0139 - val_acc: 0.9960
Epoch 7/10
54000/54000 [==============================] - 10s - loss: 0.0097 - acc: 0.9967 - val_loss: 0.0092 - val_acc: 0.9969
Epoch 8/10
54000/54000 [==============================] - 10s - loss: 0.0083 - acc: 0.9972 - val_loss: 0

[INFO:239] 2017-07-06 01:01:42,610 > Test accuracy: 99.57%
[INFO:239] 2017-07-06 01:01:42,610 > Test accuracy: 99.57%


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sequence (InputLayer)  (None, 28, 28)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               51600     
_________________________________________________________________
fc (Dense)                   (None, 100)               10100     
_________________________________________________________________
prediction (Dense)           (None, 10)                1010      
Total params: 62,710
Trainable params: 62,710
Non-trainable params: 0
_________________________________________________________________
 9824/10000 [============================>.] - ETA: 0s

[INFO:255] 2017-07-06 01:01:45,085 > Test accuracy: 99.66%
[INFO:255] 2017-07-06 01:01:45,085 > Test accuracy: 99.66%


In [1]:
## 2 LSTM Layer
## RNN 은 layer를 많이 쌓는다고 성능이 크게 향상되지 않는 경향

### Step 1: Import modules
import os
import logging

import numpy as np
import keras
import keras.backend as K

from keras.datasets import mnist
from keras.models import Model, Input, load_model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM

## Fix random seed for reproducibility
np.random.seed(20170704)


## Check proper working directory
#os.chdir('path/to/day_2/')
#if os.getcwd().split('/')[-1] == 'day_2':
#    pass
#else:
#    raise OSError('Check current working directory.\n'
#                  'If not specified as instructed, '
#                  'more errors will occur throught the code.\n'
#                  '- Current working directory: %s' % os.getcwd())
## Check proper working directory
path = os.getcwd()
os.chdir(path)
if os.getcwd().split('/')[-1] == 'DLdata':
    pass
else:
    path = os.getcwd()+'/DLdata'
    #raise OSError('Check current working directory.\n'
    #              'If not specified as instructed, '
    #              'more errors will occur throught the code.\n'
    #              '- Current working directory: %s' % os.getcwd())
print(path)


## Set logging
def set_logging(testlog=False):
    # 1. Make 'logger' instance
    logger = logging.getLogger()
    # 2. Make 'formatter'
    formatter = logging.Formatter(
            '[%(levelname)s:%(lineno)s] %(asctime)s > %(message)s'
            )
    # 3. Make 'streamHandler'
    streamHandler = logging.StreamHandler()
    # 4. Set 'formatter' to 'streamHandler'
    streamHandler.setFormatter(formatter)
    # 5. Add streamHandler to 'logger' instance
    logger.addHandler(streamHandler)
    # 6. Set level of log; DEBUG -> INFO -> WARNING -> ERROR -> CRITICAL
    logger.setLevel(logging.DEBUG)
    # 7. Print test INFO message
    if testlog: # default is 'False'
        logging.info("Stream logging available!")
    
    return logger

_ = set_logging()


####################################################################################


### Step 2: Load & preprocess data

## 2-1. Load
# Data, shuffled and split between train / test sets
# shape of X_train, X_test; (batch_size, height, width)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
logging.info('MNIST data has been loaded.')

## 2-2. Preprocess
# Change data types to 'float32'
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


# Normalization
X_train /= 255
X_test /= 255

# Check input shape of data; (batch_size, timesteps, input_dim)
timesteps = 28
input_dim = 28
#timesteps = 14
#input_dim = 56

X_train = X_train.reshape(-1, timesteps, input_dim) # -1: reshape 할때, 알아서 채우라는 의미
X_test = X_test.reshape(-1, timesteps, input_dim)

logging.info('final training data shape: {}'.format(X_train.shape))
logging.info('final test data shape: {}'.format(X_test.shape))

# Convert class vectors to binary class matrices (one-hot vectors)
num_classes = 10
Y_train = keras.utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.to_categorical(y_test, num_classes)

logging.info('final train label shape: {}'.format(y_train.shape))
logging.info('final test label shape: {}'.format(y_test.shape))


####################################################################################


### Step 3: Build Model

## 3-1. Define hyperparameters
epochs = 10
batch_size = 128
hidden_size = 100
num_classes = 10

## 3-2. Define RNN model with LSTM cells for MNIST data

# TODO: DEFINE INPUT TENSOR (hint; (timesteps, input_dim))
input_sequences = Input(shape=(timesteps, input_dim), name='input_sequence')

# TODO: DEFINE LSTM LAYER (with hidden_size=100)
x = LSTM(units=hidden_size,
         dropout=0.,
         recurrent_dropout=0.,
         kernel_initializer='glorot_uniform',
         recurrent_initializer='orthogonal',
         return_sequences=True,
         name='lstm1')(input_sequences)

x = LSTM(units=hidden_size,
         dropout=0.,
         recurrent_dropout=0.,
         kernel_initializer='glorot_uniform',
         recurrent_initializer='orthogonal',
         return_sequences=False,
         name='lstm2')(x)

# TODO: DEFINE DENSE LAYER (with hidden_size=100)
x = Dense(units=100, activation='relu', name='fc')(x)

# TODO: DEFINE SOFTMAX LAYER (10 classes)
prediction = Dense(units=10, activation='softmax', name='prediction')(x)

# INSTANTIATE MODEL
model = Model(inputs=input_sequences,
              outputs=prediction,
              name='LSTM_mnist_2')


####################################################################################


### Step 4: Define callbacks

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

# List of callbacks
callbacks = []

# Model checkpoints
ckpt_path = path+'/lstm_mnist_ckpts/lstm_mnist.{epoch:02d}-{val_acc:.2f}.hdf5'
if not os.path.exists(os.path.dirname(ckpt_path)):
    os.makedirs(os.path.dirname(ckpt_path))

checkpoint = ModelCheckpoint(filepath=ckpt_path,
                             monitor='val_acc',
                             save_best_only=True,
                             verbose=1)
callbacks.append(checkpoint)

# Stop training early
earlystopping = EarlyStopping(monitor='val_loss',
                              patience=5,
                              verbose=1)
callbacks.append(earlystopping)

# Reduce learning rate when learning does not improve
reducelr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.1, 
                             patience=10,
                             verbose=1)
callbacks.append(reducelr)

# Tensorboard for visualization; only available with tensorflow backend
# In the terminal; tensorboard --logdir='/full/path/to/lstm_mnist_logs/'
if K.backend() == 'tensorflow':
    logging.info('Using tensorboard callback')
    tb_logdir = path+'/lstm_mnist_logs/'
    if not os.path.exists(tb_logdir):
        os.makedirs(tb_logdir)
    tensorboard = TensorBoard(log_dir=tb_logdir,
                              histogram_freq=1,
                              write_graph=True)
    callbacks.append(tensorboard)


####################################################################################


### Step 5: Compile & train model

# TODO: COMPILE MODEL
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])

print(model.summary())


history = model.fit(X_train, Y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=callbacks,
                    verbose=1)


####################################################################################


### Step 6: Save & load model weights

# Save model weights
model.save_weights(path+'/weights/lstm_mnist_weights2.h5')

# Load model weights
model.load_weights(path+'/weights/lstm_mnist_weights2.h5')


####################################################################################


### Step 7: Test model performance

test_scores = model.evaluate(X_test, Y_test, verbose=1)
logging.info('Test accuracy: %.2f%%' %(test_scores[1] * 100))
#print("Test accuracy: %.2f%%" % (test_scores[1] * 100))

#train_scores = model.evaluate(X_train, y_train, verbose=1)
#print("Train accuracy: %.2f%%" % (train_scores[1] * 100))


####################################################################################


### Step 8: Using best checkpoint model

best_model_path = path+'/lstm_mnist_ckpts/lstm_mnist.08-1.00.hdf5' # must change filename
best_model = load_model(best_model_path)
best_model.summary()
test_scores = best_model.evaluate(X_test, Y_test, verbose=1)
logging.info('Test accuracy: %.2f%%' %(test_scores[1] * 100))

K.clear_session()

Using TensorFlow backend.
[INFO:77] 2017-07-06 01:14:35,755 > MNIST data has been loaded.


/home/user/DataScience/DataScience/Study Note/Deep Learning/DLdata


[INFO:98] 2017-07-06 01:14:35,867 > final training data shape: (60000, 28, 28)
[INFO:99] 2017-07-06 01:14:35,868 > final test data shape: (10000, 28, 28)
[INFO:106] 2017-07-06 01:14:35,870 > final train label shape: (60000,)
[INFO:107] 2017-07-06 01:14:35,870 > final test label shape: (10000,)
[INFO:195] 2017-07-06 01:14:36,098 > Using tensorboard callback


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sequence (InputLayer)  (None, 28, 28)            0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 28, 100)           51600     
_________________________________________________________________
lstm2 (LSTM)                 (None, 100)               80400     
_________________________________________________________________
fc (Dense)                   (None, 100)               10100     
_________________________________________________________________
prediction (Dense)           (None, 10)                1010      
Total params: 143,110
Trainable params: 143,110
Non-trainable params: 0
_________________________________________________________________
None
Train on 54000 samples, validate on 6000 samples
INFO:tensorflow:Summary name lstm1/kernel:0 is illegal; using lstm1/kernel_0 inste

[INFO:82] 2017-07-06 01:14:37,453 > Summary name lstm1/kernel:0 is illegal; using lstm1/kernel_0 instead.


...

[INFO:82] 2017-07-06 01:14:37,468 > Summary name prediction/bias:0 is illegal; using prediction/bias_0 instead.


Epoch 1/10
54000/54000 [==============================] - 19s - loss: 0.1035 - acc: 0.9619 - val_loss: 0.0361 - val_acc: 0.9872
Epoch 2/10
54000/54000 [==============================] - 19s - loss: 0.0301 - acc: 0.9896 - val_loss: 0.0226 - val_acc: 0.9921
Epoch 3/10
54000/54000 [==============================] - 19s - loss: 0.0192 - acc: 0.9937 - val_loss: 0.0145 - val_acc: 0.9957
Epoch 4/10
54000/54000 [==============================] - 21s - loss: 0.0139 - acc: 0.9954 - val_loss: 0.0098 - val_acc: 0.9968
Epoch 5/10
54000/54000 [==============================] - 19s - loss: 0.0110 - acc: 0.9964 - val_loss: 0.0117 - val_acc: 0.9962
Epoch 6/10
54000/54000 [==============================] - 19s - loss: 0.0093 - acc: 0.9970 - val_loss: 0.0114 - val_acc: 0.9964
Epoch 7/10
54000/54000 [==============================] - 19s - loss: 0.0076 - acc: 0.9975 - val_loss: 0.0098 - val_acc: 0.9972
Epoch 8/10
54000/54000 [==============================] - 19s - loss: 0.0066 - acc: 0.9978 - val_loss: 0

[INFO:244] 2017-07-06 01:17:59,147 > Test accuracy: 99.73%



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sequence (InputLayer)  (None, 28, 28)            0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 28, 100)           51600     
_________________________________________________________________
lstm2 (LSTM)                 (None, 100)               80400     
_________________________________________________________________
fc (Dense)                   (None, 100)               10100     
_________________________________________________________________
prediction (Dense)           (None, 10)                1010      
Total params: 143,110
Trainable params: 143,110
Non-trainable params: 0
_________________________________________________________________
 9856/10000 [============================>.] - ETA: 0s

[INFO:260] 2017-07-06 01:18:03,714 > Test accuracy: 99.71%
